<a href="https://colab.research.google.com/github/xuanhiennb/Mechanisms-of-Action/blob/main/LightGBM_bagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone --recursive https://github.com/Microsoft/LightGBM

Cloning into 'LightGBM'...
remote: Enumerating objects: 20573, done.
remote: Total 20573 (delta 0), reused 0 (delta 0), pack-reused 20573
Receiving objects: 100% (20573/20573), 16.01 MiB | 4.88 MiB/s, done.
Resolving deltas: 100% (15026/15026), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'compute'
Cloning into '/content/LightGBM/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728        
Receiving objects: 100% (21728/21728), 8.51 MiB | 10.27 MiB/s, done.
Resolving deltas: 100% (17565/17565), done.
Submodule path 'compute': checked out '36c89134d4013b2e5e45bc55656a18bd6141995a'


In [ ]:
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;  

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Looking for CL_VERSION_2_2
-- Looking for CL_VERSION_2_2 - found
-- Found OpenCL: /usr/lib/x86_64-linux-gnu/libOpenCL.so (found version "2.2") 
-- OpenCL include directory: /usr/include
-- Boost version: 1.65.1
-- Found the following Boost libraries:
--

In [ ]:
import lightgbm as lgb

### Getting feats importance

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('../input/lish-moa/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Read data

In [ ]:
folder_path = '/content/drive/My Drive/Colab Notebooks/'
input_path = folder_path #'../input/dictfeats/'


In [ ]:

train = pd.read_csv(folder_path +'train_features.csv')

train.shape

In [ ]:
train_target = pd.read_csv(folder_path +'train_targets_scored.csv')

train_target.shape

In [ ]:
test = pd.read_csv(folder_path +'test_features.csv')

test.shape

## Check agonist / antagonist

In [ ]:
antagonists = []

for column in train_target.columns:
    if column.endswith('_agonist'):
        antagonist = column.replace('_agonist', '_antagonist')
        if antagonist in train_target.columns:
            antagonists.append((column, antagonist))

antagonists

List agonist / antagonist pairs where both features have 1

In [ ]:
for pair in antagonists:
    n = train_target[(train_target[pair[0]] == 1) & (train_target[pair[1]] == 1)].shape[0]
    if n > 0:
        print(pair[0], '-', pair[1])
        print('Number of cases:', n)

In general agonist and antagonist should not have 1 at the same time

## Feature engineering

In [ ]:

train.at[train['cp_type'].str.contains('ctl_vehicle'),train.filter(regex='-.*').columns] = 0.0

test.at[test['cp_type'].str.contains('ctl_vehicle'),test.filter(regex='-.*').columns] = 0.0

## OHE

In [ ]:
train_size = train.shape[0]

traintest = pd.concat([train, test])

traintest = pd.concat([traintest, pd.get_dummies(traintest['cp_type'], prefix='cp_type')], axis=1)
traintest = pd.concat([traintest, pd.get_dummies(traintest['cp_time'], prefix='cp_time')], axis=1)
traintest = pd.concat([traintest, pd.get_dummies(traintest['cp_dose'], prefix='cp_dose')], axis=1)

traintest = traintest.drop(['cp_type', 'cp_time', 'cp_dose'], axis=1)

train = traintest[:train_size]
test  = traintest[train_size:]

del traintest

In [ ]:
train.shape

## Prepare x, y

In [ ]:
X_1 = train.drop('sig_id', axis=1)

Y_1 = train_target.drop('sig_id', axis=1)

X_final = test.drop('sig_id', axis=1)

## Feature selection

In [ ]:
# from https://www.kaggle.com/pavelvpster/moa-keras-optuna-rfecv

# 500
#selected_features = #['g-0', 'g-1', 'g-2', 'g-3', 'g-4', 'g-5', 'g-6', 'g-7', 'g-8', 'g-9', 'g-10', 'g-11', 'g-12', 'g-13', 'g-14', 'g-15', 'g-16', 'g-17', 'g-18', 'g-19', 'g-20', 'g-21', 'g-22', 'g-23', 'g-24', 'g-25', 'g-26', 'g-27', 'g-28', 'g-29', 'g-30', 'g-31', 'g-32', 'g-33', 'g-34', 'g-35', 'g-36', 'g-37', 'g-38', 'g-39', 'g-40', 'g-41', 'g-42', 'g-43', 'g-44', 'g-45', 'g-46', 'g-47', 'g-48', 'g-49', 'g-50', 'g-51', 'g-52', 'g-53', 'g-54', 'g-55', 'g-56', 'g-57', 'g-58', 'g-59', 'g-60', 'g-61', 'g-62', 'g-63', 'g-64', 'g-65', 'g-66', 'g-67', 'g-68', 'g-69', 'g-70', 'g-71', 'g-72', 'g-73', 'g-74', 'g-75', 'g-76', 'g-77', 'g-78', 'g-79', 'g-80', 'g-81', 'g-82', 'g-83', 'g-84', 'g-85', 'g-86', 'g-87', 'g-88', 'g-89', 'g-90', 'g-91', 'g-92', 'g-93', 'g-94', 'g-95', 'g-96', 'g-97', 'g-98', 'g-99', 'g-100', 'g-101', 'g-102', 'g-103', 'g-104', 'g-105', 'g-106', 'g-107', 'g-108', 'g-109', 'g-110', 'g-111', 'g-112', 'g-113', 'g-114', 'g-115', 'g-116', 'g-117', 'g-118', 'g-121', 'g-122', 'g-123', 'g-125', 'g-126', 'g-127', 'g-128', 'g-129', 'g-130', 'g-131', 'g-132', 'g-133', 'g-139', 'g-144', 'g-149', 'g-152', 'g-153', 'g-154', 'g-155', 'g-158', 'g-161', 'g-162', 'g-165', 'g-166', 'g-167', 'g-168', 'g-169', 'g-170', 'g-171', 'g-172', 'g-173', 'g-174', 'g-175', 'g-176', 'g-177', 'g-178', 'g-179', 'g-180', 'g-182', 'g-185', 'g-188', 'g-189', 'g-191', 'g-192', 'g-193', 'g-194', 'g-195', 'g-196', 'g-197', 'g-198', 'g-199', 'g-200', 'g-201', 'g-202', 'g-203', 'g-204', 'g-209', 'g-211', 'g-213', 'g-214', 'g-217', 'g-220', 'g-221', 'g-222', 'g-223', 'g-225', 'g-226', 'g-229', 'g-233', 'g-239', 'g-240', 'g-243', 'g-244', 'g-246', 'g-247', 'g-250', 'g-254', 'g-255', 'g-256', 'g-257', 'g-259', 'g-261', 'g-262', 'g-265', 'g-269', 'g-270', 'g-271', 'g-273', 'g-279', 'g-282', 'g-284', 'g-286', 'g-289', 'g-292', 'g-294', 'g-295', 'g-298', 'g-299', 'g-300', 'g-303', 'g-306', 'g-310', 'g-311', 'g-312', 'g-314', 'g-316', 'g-317', 'g-318', 'g-320', 'g-321', 'g-323', 'g-326', 'g-327', 'g-328', 'g-332', 'g-333', 'g-335', 'g-339', 'g-340', 'g-341', 'g-346', 'g-347', 'g-351', 'g-353', 'g-355', 'g-357', 'g-360', 'g-364', 'g-365', 'g-370', 'g-375', 'g-383', 'g-385', 'g-388', 'g-389', 'g-390', 'g-391', 'g-392', 'g-399', 'g-401', 'g-402', 'g-403', 'g-404', 'g-409', 'g-417', 'g-423', 'g-424', 'g-425', 'g-426', 'g-428', 'g-429', 'g-430', 'g-431', 'g-433', 'g-434', 'g-438', 'g-439', 'g-440', 'g-441', 'g-443', 'g-444', 'g-445', 'g-446', 'g-447', 'g-448', 'g-449', 'g-450', 'g-451', 'g-452', 'g-453', 'g-454', 'g-455', 'g-456', 'g-460', 'g-461', 'g-463', 'g-465', 'g-467', 'g-470', 'g-473', 'g-474', 'g-475', 'g-476', 'g-481', 'g-483', 'g-485', 'g-486', 'g-489', 'g-490', 'g-495', 'g-500', 'g-501', 'g-502', 'g-504', 'g-509', 'g-513', 'g-516', 'g-517', 'g-518', 'g-527', 'g-531', 'g-535', 'g-540', 'g-542', 'g-543', 'g-544', 'g-547', 'g-548', 'g-549', 'g-552', 'g-556', 'g-558', 'g-562', 'g-566', 'g-571', 'g-572', 'g-573', 'g-574', 'g-578', 'g-584', 'g-585', 'g-590', 'g-592', 'g-595', 'g-596', 'g-600', 'g-602', 'g-603', 'g-605', 'g-606', 'g-611', 'g-612', 'g-615', 'g-617', 'g-618', 'g-619', 'g-620', 'g-623', 'g-624', 'g-625', 'g-626', 'g-627', 'g-628', 'g-629', 'g-630', 'g-631', 'g-632', 'g-633', 'g-634', 'g-635', 'g-636', 'g-637', 'g-638', 'g-639', 'g-640', 'g-643', 'g-645', 'g-646', 'g-648', 'g-651', 'g-654', 'g-657', 'g-658', 'g-659', 'g-660', 'g-661', 'g-662', 'g-663', 'g-664', 'g-665', 'g-666', 'g-668', 'g-670', 'g-672', 'g-673', 'g-674', 'g-675', 'g-677', 'g-678', 'g-679', 'g-685', 'g-689', 'g-695', 'g-697', 'g-698', 'g-699', 'g-701', 'g-702', 'g-703', 'g-704', 'g-707', 'g-709', 'g-710', 'g-712', 'g-713', 'g-714', 'g-715', 'g-721', 'g-722', 'g-724', 'g-725', 'g-726', 'g-734', 'g-736', 'g-737', 'g-740', 'g-742', 'g-743', 'g-745', 'g-747', 'g-750', 'g-751', 'g-752', 'g-753', 'g-756', 'g-757', 'g-758', 'g-759', 'g-760', 'g-761', 'g-762', 'g-763', 'g-764', 'g-766', 'g-767', 'g-768', 'g-769', 'g-770', 'g-771', 'c-0', 'c-1', 'c-2', 'c-3', 'c-4', 'c-5', 'c-11', 'c-12', 'c-14', 'c-19', 'c-20', 'c-21', 'c-22', 'c-23', 'c-25', 'c-27', 'c-32', 'c-35', 'c-36', 'c-37', 'c-40', 'c-47', 'c-54', 'c-55', 'c-57', 'c-58', 'c-61', 'c-62', 'c-63', 'c-65', 'c-66', 'c-71', 'c-74', 'c-75', 'c-77', 'c-78', 'c-79', 'c-80', 'c-81', 'c-82', 'c-83', 'c-84', 'c-85', 'c-86', 'c-87', 'c-88', 'c-89', 'c-90', 'c-91', 'c-92', 'c-93', 'c-97']

In [ ]:
# x_train = x_train[selected_features]

# x_test = x_test[selected_features]

## LightGBM

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import log_loss

import lightgbm as lgb

In [ ]:
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import RandomUnderSampler 
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from matplotlib import pyplot
from numpy import where
def resampling_data(X,y):
      counter = Counter(y)
      print(counter)
      # define pipeline
      over = SMOTE(sampling_strategy=0.2)
      under = RandomUnderSampler(sampling_strategy=1)
      steps = [('o', over), ('u', under)]
      pipeline = Pipeline(steps=steps)
      # transform the dataset
      X, y = pipeline.fit_resample(X, y)
      # summarize the new class distribution
      counter = Counter(y)
      print(counter)
      # scatter plot of examples by class label
      for label, _ in counter.items():
        row_ix = where(y == label)[0]
        pyplot.scatter(X[row_ix, 0], X[row_ix, 1], label=str(label))
      pyplot.legend()
      pyplot.show()
      return X,y

In [ ]:
def model_sub(params,X_train,y_train,X_test,y_test):
    print("model")
    clf = lgb.LGBMClassifier(
                n_estimators=100,
                learning_rate=0.01,
                n_jobs=20,
               # max_depth=30,
                random_state=2020
               # num_leaves= 50
        )
    clf.fit(
        X_train,y_train, 
        eval_set= [(X_train,y_train), (X_test, y_test)], 
        eval_metric='log_loss', verbose=-1, early_stopping_rounds=300  #30
    )
    score= clf.best_score_['valid_1']['binary_logloss']
    return clf,score
    

In [ ]:
p_min = 0.001
p_max = 0.999
def logloss(y_true, y_pred):
    y_pred = np.clip(y_pred,p_min,p_max)
    return -np.mean(y_true*np.log(y_pred) + (1-y_true)*np.log(1-y_pred))

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_1,Y_1,test_size=0.3,random_state=2020)

In [ ]:
submission = pd.read_csv(folder_path +'sample_submission.csv')

In [ ]:
target_cols = ['dopamine_receptor_antagonist']


In [ ]:
# import pickle

# # open a file, where you stored the pickled data
# params_model={}
# filename=param_path+"full_params_tuned.txt"
# file = open(filename, 'rb')
# # dump information to that file
# params_tune = pickle.load(file)
# params_model=params_tune
# # close the file
# file.close()
# params_model

In [ ]:
import pickle
input_feats_file=folder_path +'feats_important_v2.txt'
with open(input_feats_file, 'rb') as handle:
  dict_feats = pickle.loads(handle.read())

In [ ]:
import pickle
filename = folder_path+"feats_important_v3.txt"
pickle.dump(dict_feats, open(filename, 'wb'))

In [ ]:
def model_sub_1(params,X_train,y_train,X_test,y_test,state):
    dtrain = lgb.Dataset(X_train, label=y_train)
    dval = lgb.Dataset(X_test, label=y_test)
    print("model")
    params={
            'objective': 'binary',
            'metric': 'binary_logloss',
            'boosting_type': 'gbdt',
            'boost_from_average': True,
            'num_threads': 20,
            'random_state': state,

            'learning_rate': 0.01,
            'num_iterations':1000,

            # from Optuna result in Version 7
            'num_leaves': 100 ,
            'min_data_in_leaf': 100,
            'min_child_weight': 0.0010123391323415569,
            'max_depth': 10,
            'bagging_fraction': 0.7968351296815959,
            'feature_fraction': 0.8556374471450119,
            'lambda_l1': 0.23497601594060086,
            'lambda_l2': 0.15889208239516134,
            'device': 'gpu',
            'gpu_platform_id': 0,
            'gpu_device_id': 0,
            'max_bin':63
    }
    print(params)
    model = lgb.train(
        params, dtrain, valid_sets=[dtrain, dval], verbose_eval=-1, early_stopping_rounds=200
    )
    y_pred=model.predict(X_test)
    score= logloss(y_test,y_pred)
    return model,score

In [ ]:
target_cols=['retinoid_receptor_agonist', 'retinoid_receptor_antagonist', 'rho_associated_kinase_inhibitor', 'ribonucleoside_reductase_inhibitor', 'rna_polymerase_inhibitor', 'serotonin_receptor_agonist', 'serotonin_receptor_antagonist', 'serotonin_reuptake_inhibitor', 'sigma_receptor_agonist', 'sigma_receptor_antagonist', 'smoothened_receptor_antagonist', 'sodium_channel_inhibitor', 'sphingosine_receptor_agonist', 'src_inhibitor', 'steroid', 'syk_inhibitor', 'tachykinin_antagonist', 'tgf-beta_receptor_inhibitor', 'thrombin_inhibitor', 'thymidylate_synthase_inhibitor', 'tlr_agonist', 'tlr_antagonist', 'tnf_inhibitor', 'topoisomerase_inhibitor', 'transient_receptor_potential_channel_antagonist', 'tropomyosin_receptor_kinase_inhibitor', 'trpv_agonist', 'trpv_antagonist', 'tubulin_inhibitor', 'tyrosine_kinase_inhibitor', 'ubiquitin_specific_protease_inhibitor', 'vegfr_inhibitor', 'vitamin_b', 'vitamin_d_receptor_agonist', 'wnt_inhibitor']

In [ ]:
def get_top_feats(clf,X_train):
    features = X_train.columns
    importances = clf.feature_importance()
    indices = np.argsort(importances)[::-1]
    list_importances = []
    for i in range(X_train.shape[1]):
        if importances[indices[i]] > 0:
            list_importances.append(features[indices[i]])
    return list_importances

In [ ]:
submission =pd.DataFrame() 

In [ ]:
import pickle
def save_model(model,col,i):
  filename = folder_path+"/moa_models_1/"+col+"_"+str(i)+".model"
  pickle.dump(model, open(filename, 'wb'))

In [ ]:
scores = []
feats_dict={'':[]}
feats_num={'': 1}
score_model={'':1}
params={}
count=0
for col in target_cols:
    print(col)
    count=count+10
    list_feats = dict_feats[col]
    temp_score=pd.DataFrame()
    X_train,X_test,y_train,y_test=train_test_split(X_1,Y_1,test_size=0.3,random_state=1000)
    X_train_res,y_train_res=X_train[list_feats],y_train[col]
    clf,score= model_sub_1(params,X_train_res,y_train_res,X_test[list_feats],y_test[col],1000)
    min_score=score
    model=clf
    for i in range(25):
      X_train,X_test,y_train,y_test=train_test_split(X_1,Y_1,test_size=0.3,random_state=(count+i+2000))
      X_train_res,y_train_res=X_train[list_feats],y_train[col]
      clf,score= model_sub_1(params,X_train_res,y_train_res,X_test[list_feats],y_test[col],(count+i+2000))
      save_model(model,col,i)
    score_model[col]=min_score
    scores.append(min_score)
    test_pred=model.predict(X_final[list_feats])
    y_pred_final=test_pred 
    submission[col]=y_pred_final  
    print(min_score)
print(np.mean(scores))

In [ ]:
!pip install cmake

In [ ]:
score_model

In [ ]:
submission[col]

In [ ]:
file_out=folder_path+'/moa_feats/LBM_feats.csv'
submission.to_csv(file_out, index=False)